In [10]:
!pip install qiskit

In [ ]:
# unzipped already, no need to do again
!unzip pima-indians-diabetes-database.zip

pima-indians-diabetes-database.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  pima-indians-diabetes-database.zip
replace diabetes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# just in case someone wants to see the ugly comma seperated file
import csv
with open('/data/diabetes.csv', newline='') as File:  
    reader = csv.reader(File)
    for row in reader:
        print(row)

In [11]:
import numpy as np
data = np.genfromtxt('diabetes.csv', delimiter=',', names=True, case_sensitive=True)
data = np.array(data.tolist()) # was an array of arrays earlier
print(data.shape)
x = np.copy(data[:,0:8]) #features
y = np.copy(data[:,8]) #targets
features = 8 #or number of qubits which I kept equal to number of features in this case

(768, 9)


In [8]:
from sklearn.model_selection import train_test_split
import qiskit
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import one_against_rest, all_pairs
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
#from qiskit.aqua.input import ClassificationInput
#from qiskit.aqua import run_algorithm

In [12]:
#please and please again, use your own IBMQ token (can be generated easily), I have very few processes left for me.
key = '243ff4202efc1b45ad9282b9d6d524cdcccfc25561a86de37424b5ddd9aaa72969ea300500423019c48da10777e3241a5a45ff4a223439239cba4bed8d8709cc'

In [13]:
qiskit.IBMQ.save_account(key)
provider = qiskit.IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

configrc.store_credentials:WARNING:2020-06-16 03:17:14,078: Credentials already present. Set overwrite=True to overwrite.


In [26]:
#using only first 100, if anyone is brave enough, train_test_split(x, y, test_size=0.2)
train_x, test_x, train_y, test_y = train_test_split(x[0:301], y[0:301], test_size=0.2)

training_data = {'A':train_x[train_y==0],'B':train_x[train_y==1]}
testing_data = {'A':test_x[test_y==0],'B':test_x[test_y==1]}

In [27]:
#Using RawFeatureVector reduces time immensely!
#feature_map = SecondOrderExpansion(feature_dimension=features,depth=2,entanglement='full')
from qiskit.aqua.components.feature_maps import RawFeatureVector
feature_map = RawFeatureVector(8)

svm = QSVM(feature_map,training_data,testing_data)
shots = 1 
quantum_instance = QuantumInstance(backend,shots=shots,skip_qobj_validation=False)

In [13]:
!pip install cvxopt

     |████████████████████████████████| 11.6 MB 6.2 MB/s eta 0:00:01


In [29]:
%%time
result = svm.run(quantum_instance) 

CPU times: user 11min 52s, sys: 1min 15s, total: 13min 8s
Wall time: 57min 1s


In [30]:
print(result)

{'kernel_matrix_training': array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.]]), 'svm': {'alphas': array([212.87566143, 174.9973143 ,  32.64708417, 589.84893371,
       187.12569218, 342.80960092, 297.47077578,  33.30720946,
       215.57268412,   2.31291416, 437.72892966, 157.69552339,
       475.91710585,  31.46983758, 128.37870771,  28.65463845,
       189.78494108, 186.11149126, 124.37150644, 121.95860059,
        39.44129889,  41.24034777,  99.54584377, 390.49443017,
       164.27688675, 117.6670089 ,  93.04434275, 347.07495768,
       216.73040109,  41.17016827, 298.76906763,  19.17014655,
       113.70103817, 133.82383186, 417.82963409, 197.17034712,
       140.43209356,  12.01317432,  21.43539347, 224.4768984 ,
       200.19270518, 131.06655371,  89.32790178, 127.79618571,
        97.9765247 ,